In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=8c407bd7d50a8031c9d5388a4d58d1a086159618bb21419cbd722e22482e4f9e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/29 16:02:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
train_df = spark.read.csv('/kaggle/input/leash-BELKA/train.csv')

In [9]:
test_df = spark.read.csv('/kaggle/input/leash-BELKA/test.csv')

In [10]:
train_df.show()

+---+--------------------+--------------------+--------------------+--------------------+------------+-----+
|_c0|                 _c1|                 _c2|                 _c3|                 _c4|         _c5|  _c6|
+---+--------------------+--------------------+--------------------+--------------------+------------+-----+
| id|buildingblock1_sm...|buildingblock2_sm...|buildingblock3_sm...|     molecule_smiles|protein_name|binds|
|  0|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|        BRD4|    0|
|  1|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         HSA|    0|
|  2|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         sEH|    0|
|  3|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|   Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|        BRD4|    0|
|  4|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|   Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|         HSA|    0|
|  5|C#CC[C@@H](CC(

In [11]:
train_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



In [12]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

In [13]:
data_schema = [StructField("_c0", IntegerType(), True),
               StructField("_c1", StringType(), True),
              StructField("_c2", StringType(), True),
              StructField("_c3", StringType(), True),
              StructField("_c4", StringType(), True),
              StructField("_c5", StringType(), True),
              StructField("_c6", IntegerType(), True)]
final_struc = StructType(fields=data_schema)
train_df = spark.read.csv('/kaggle/input/leash-BELKA/train.csv', schema=final_struc)
train_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: integer (nullable = true)



In [14]:
train_df.columns

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6']

In [15]:
train_df.show()

+----+--------------------+--------------------+--------------------+--------------------+------------+----+
| _c0|                 _c1|                 _c2|                 _c3|                 _c4|         _c5| _c6|
+----+--------------------+--------------------+--------------------+--------------------+------------+----+
|NULL|buildingblock1_sm...|buildingblock2_sm...|buildingblock3_sm...|     molecule_smiles|protein_name|NULL|
|   0|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|        BRD4|   0|
|   1|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         HSA|   0|
|   2|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         sEH|   0|
|   3|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|   Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|        BRD4|   0|
|   4|C#CC[C@@H](CC(=O)...|C#CCOc1ccc(CN)cc1.Cl|   Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|         HSA|   0|
|   5|C#CC[C@@H](CC

In [16]:
train_df.head(5)

[Row(_c0=None, _c1='buildingblock1_smiles', _c2='buildingblock2_smiles', _c3='buildingblock3_smiles', _c4='molecule_smiles', _c5='protein_name', _c6=None),
 Row(_c0=0, _c1='C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21', _c2='C#CCOc1ccc(CN)cc1.Cl', _c3='Br.Br.NCC1CCCN1c1cccnn1', _c4='C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1', _c5='BRD4', _c6=0),
 Row(_c0=1, _c1='C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21', _c2='C#CCOc1ccc(CN)cc1.Cl', _c3='Br.Br.NCC1CCCN1c1cccnn1', _c4='C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1', _c5='HSA', _c6=0),
 Row(_c0=2, _c1='C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21', _c2='C#CCOc1ccc(CN)cc1.Cl', _c3='Br.Br.NCC1CCCN1c1cccnn1', _c4='C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1', _c5='sEH', _c6=0),
 Row(_c0=3, _c1='C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21', _c2='C#CCOc1ccc(CN)cc1.Cl', _c3='Br.NCc1cccc(Br)n1', _c4='C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)

In [17]:
print(train_df['_c0'])

Column<'_c0'>


In [18]:
train_df = train_df.withColumnRenamed("_c0", "id")

In [20]:
train_df.show(1)

+----+--------------------+--------------------+--------------------+---------------+------------+----+
|  id|                 _c1|                 _c2|                 _c3|            _c4|         _c5| _c6|
+----+--------------------+--------------------+--------------------+---------------+------------+----+
|NULL|buildingblock1_sm...|buildingblock2_sm...|buildingblock3_sm...|molecule_smiles|protein_name|NULL|
+----+--------------------+--------------------+--------------------+---------------+------------+----+
only showing top 1 row



In [21]:
train_df = train_df.withColumnRenamed("_c1", "buildingblock1_smiles")
train_df = train_df.withColumnRenamed("_c2", "buildingblock2_smiles")
train_df = train_df.withColumnRenamed("_c3", "buildingblock3_smiles")
train_df = train_df.withColumnRenamed("_c4", "molecule_smiles")
train_df = train_df.withColumnRenamed("_c5", "protein_name")
train_df = train_df.withColumnRenamed("_c6", "binds")

In [25]:
train_df.show(5)

+----+---------------------+---------------------+---------------------+--------------------+------------+-----+
|  id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|binds|
+----+---------------------+---------------------+---------------------+--------------------+------------+-----+
|NULL| buildingblock1_sm...| buildingblock2_sm...| buildingblock3_sm...|     molecule_smiles|protein_name| NULL|
|   0| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl| Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|        BRD4|    0|
|   1| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl| Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         HSA|    0|
|   2| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl| Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         sEH|    0|
|   3| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl|    Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|        BRD4|    0|
+----+---------------------+---------------------+---------------------+--------------------+---

In [27]:
train_df.select('id').show()

+----+
|  id|
+----+
|NULL|
|   0|
|   1|
|   2|
|   3|
|   4|
|   5|
|   6|
|   7|
|   8|
|   9|
|  10|
|  11|
|  12|
|  13|
|  14|
|  15|
|  16|
|  17|
|  18|
+----+
only showing top 20 rows



In [44]:
train_df = train_df.filter(train_df.id >= 0)

In [45]:
train_df.show()

+---+---------------------+---------------------+---------------------+--------------------+------------+-----+
| id|buildingblock1_smiles|buildingblock2_smiles|buildingblock3_smiles|     molecule_smiles|protein_name|binds|
+---+---------------------+---------------------+---------------------+--------------------+------------+-----+
|  0| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl| Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|        BRD4|    0|
|  1| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl| Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         HSA|    0|
|  2| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl| Br.Br.NCC1CCCN1c1...|C#CCOc1ccc(CNc2nc...|         sEH|    0|
|  3| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl|    Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|        BRD4|    0|
|  4| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl|    Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|         HSA|    0|
|  5| C#CC[C@@H](CC(=O)...| C#CCOc1ccc(CN)cc1.Cl|    Br.NCc1cccc(Br)n1|C#CCOc1ccc(CNc2nc...|         sEH

In [46]:
train_df.count()

295246830

In [48]:
train_df.createOrReplaceTempView("train")

In [52]:
sql_query = f"SELECT buildingblock1_smiles FROM train WHERE id = 0"
mol_1 = spark.sql(sql_query)

In [54]:
mol_1.first()

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


Row(buildingblock1_smiles='C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21')

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


In [5]:
mol_1_smiles = 'C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21'

In [4]:
!pip install molSimplify

In [6]:
from molSimplify.Classes.mol3D import mol3D

In [7]:
mol_1_3d = mol3D()  

In [8]:
mol_1_3d.read_smiles(mol_1_smiles)

In [9]:
print(mol_1_3d.writexyz('', writestring=True))

45
04/29/2024 17:12, XYZ structure generated by mol3D Class, molSimplify
C 	1.052641	0.060233	0.060008
C 	-0.251215	0.060233	0.060008
C 	-1.771215	0.060233	0.060008
C 	-2.295216	-0.756320	1.229436
C 	-1.839456	-0.146241	2.563035
C 	-2.302904	1.261950	2.781903
O 	-3.085393	1.620893	3.658395
O 	-1.810969	2.160335	1.920016
N 	-3.741447	-0.905446	1.080898
C 	-4.419857	-2.027806	0.672014
O 	-5.612177	-1.982003	0.393509
O 	-3.700360	-3.169892	0.790865
C 	-4.460626	-4.402052	0.804922
C 	-5.344382	-4.648584	2.066922
C 	-5.139843	-6.068250	2.638722
C 	-5.326899	-7.301367	2.041330
C 	-5.065577	-8.455756	2.790667
C 	-4.641662	-8.365664	4.125402
C 	-4.454420	-7.120750	4.731085
C 	-4.700210	-5.977174	3.984656
C 	-4.596286	-4.588964	4.363192
C 	-4.195641	-4.000802	5.552690
C 	-4.179859	-2.603264	5.659459
C 	-4.627288	-1.810175	4.608451
C 	-5.125093	-2.414315	3.458989
C 	-5.021178	-3.788984	3.290330
H 	2.118540	0.061406	0.058076
H 	-2.173084	1.073285	-0.002529
H 	-2.088980	-0.409979	-0.881485
H 	-1.8

In [11]:
!pip install py3Dmol

In [12]:
from molSimplify.Informatics.jupyter_vis import view_structures
view_structures([mol_1_3d])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.